In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
forest_fires = fetch_ucirepo(id=162) 
  
# data (as pandas dataframes) 
X = forest_fires.data.features 
y = forest_fires.data.targets 
  
# metadata 
# print(forest_fires.metadata) 
  
# variable information 
# print(forest_fires.variables) 


In [8]:
import pandas as pd
X_ohe = pd.get_dummies(X, drop_first=True)
X_ohe

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_may,month_nov,month_oct,month_sep,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,False,False,False,False,False,False,False,False,False,False
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,False,False,True,False,False,False,False,False,True,False
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,False,False,True,False,False,True,False,False,False,False
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,False,False,False,False,False,False,False,False,False,False
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,False,False,False,False,False,False,True,False,False,False
513,2,4,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,False,False,False,False,False,False,True,False,False,False
514,7,4,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,False,False,False,False,False,False,True,False,False,False
515,1,4,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,False,False,False,False,False,True,False,False,False,False


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.3, random_state=24)

In [24]:
### Finding the best combination of alpha and l1_ratio
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet
alpha=[0.01, 0.1, 0.5, 1, 2.5, 3, 5]
ratios=[0.001, 0.3, 0.5, 0.7, 0.9]
scores = []
for i in alpha :
    for j in ratios :
        e1 = ElasticNet(alpha=i, l1_ratio=j)
        e1.fit(X_train, y_train)
        y_pred = e1.predict(X_test)
        scores.append([i, j, r2_score(y_test, y_pred)])

In [13]:
df_scores=pd.DataFrame(scores, columns=['alpha', 'l1_score', 'r2'])
best_combo = df_scores.sort_values('r2', ascending=False).iloc[0]

Best alpha : 0.01
Best l1_ratio : 0.9
Best r2_score : 0.0083


### Apply `ln(1+y)` to y

In [29]:
import numpy as np
y_trn_ln = np.log1p(y_train)
e1 = ElasticNet(alpha=i, l1_ratio=j)
e1.fit(X_train, y_trn_ln)
y_pred = e1.predict(X_test)
y_pred = np.expm1(y_pred)
r2_score(y_test, y_pred)

-0.02638498218973151

In [27]:
y_pred

array([16.24796621,  8.075095  ,  5.25249068, 18.99375176,  6.08328705,
       12.95610827,  8.90746758, 19.4001615 ,  6.22302628,  4.73609206,
        5.10515261,  7.50528411,  8.9958682 ,  7.434465  ,  9.46945471,
       11.18680335, 11.78249301,  8.23105821, 13.31729581, 12.11383331,
        8.95062954,  9.80255253, 10.75178439, 17.91985276, 20.78863806,
        9.87292768, 17.49666092,  7.58090556,  5.47866769, 10.42687677,
        6.82364748, 10.7243655 , 11.40346983, 14.66684327,  6.58907095,
       16.98022778, 11.35803119, 13.75966361,  6.01644846,  7.60936544,
       10.30856499,  5.6484136 , 15.32129242,  7.86201197, 17.59403088,
        5.18031109,  8.46736943, 12.46367603,  7.07849772,  9.23368512,
        3.38493344, 12.67346087,  8.68038604, 19.92750235,  8.16084882,
        8.81214413, 12.91184539,  8.78466171,  9.15777332, 22.32471553,
       12.5861524 , 10.54583453, 16.53960932,  8.36685339, 16.9038247 ,
        3.48761572, 17.45859948,  7.96730423, 11.4310875 , 10.07